In [2]:
import os
import cv2 as cv
import numpy as np

# pip install opencv-contrib-python

Useful Function

In [3]:
def rescale(img, scale=0.5):
    width = int(img.shape[1] * scale)
    height = int(img.shape[0] * scale)
    return cv.resize(img, (width, height), interpolation=cv.INTER_AREA)

def gray(img):
    return cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Face Detection (using in-built models)

In [4]:
# Training on face data
data = cv.CascadeClassifier('Data/faceData.xml')


Images

In [5]:
img = cv.imread("Data/Images/testFaces/find4.jpeg")

# Haar Cascade [ Not very accurate (Noise prone), but easy and fast ]
faces = data.detectMultiScale(gray(img), 1.1, 2)

for (x1, y1, x2, y2) in faces:
    cv.rectangle(img, (x1, y1), (x1+x2, y1+y2), (0, 0, 255), 2)

cv.imshow('Detected Faces', img)
if cv.waitKey(0) & 0xFF == ord('q'):
    cv.destroyAllWindows()

Videos

In [6]:
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    faces = data.detectMultiScale(gray(frame), 1.1, 2)

    for (x1, y1, x2, y2) in faces:
        cv.rectangle(frame, (x1, y1), (x1+x2, y1+y2), (0, 0, 255), 2)

    cv.imshow('Detected Faces', frame)

    if cv.waitKey(33) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

# Face Recognition

Extract (and labelling) Training Data

In [9]:
features, labels = [], []
l_ctr = 0
haar = cv.CascadeClassifier('Data/faceData.xml')

dir = 'Data/Images/trainFaces'
for person in os.listdir(dir):
    path = os.path.join(dir, person)

    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = gray(rescale(cv.imread(img_path), 2))
    
        detect = haar.detectMultiScale(img, 1.1, 3)
        for (x1, y1, x2, y2) in detect:
            face = img[y1:y1+y2, x1:x1+x2]
            features.append(face)
            labels.append(l_ctr)
    l_ctr += 1

print(features, labels)

[array([[255, 251, 251, ..., 248, 248, 254],
       [255, 251, 251, ..., 248, 248, 254],
       [255, 241, 241, ..., 247, 247, 254],
       ...,
       [254, 252, 252, ..., 255, 255, 255],
       [255, 253, 253, ..., 254, 254, 254],
       [255, 253, 253, ..., 254, 254, 254]], dtype=uint8), array([[122, 143, 143, ..., 244, 119, 119],
       [118, 154, 154, ..., 167,  90,  90],
       [118, 154, 154, ..., 167,  90,  90],
       ...,
       [161, 176, 176, ...,  22,  30,  30],
       [167, 166, 166, ...,  29,  31,  31],
       [167, 166, 166, ...,  29,  31,  31]], dtype=uint8), array([[ 11,  11,  17, ...,  14,  10,  10],
       [ 11,  11,  17, ...,  14,  10,  10],
       [  8,   8,  11, ...,  36,  22,  22],
       ...,
       [ 12,  12,  12, ..., 150,  81,  81],
       [ 12,  12,  12, ...,  75, 109, 109],
       [ 12,  12,  12, ...,  75, 109, 109]], dtype=uint8), array([[186, 186, 211, ..., 233, 233, 254],
       [106, 106, 107, ..., 254, 254, 254],
       [106, 106, 107, ..., 254, 254, 

Training (and Saving) Model

In [10]:
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.train(np.array(features, dtype=object), np.array(labels))

face_recognizer.save('Data/face_trained.yml')
# np.save('OpenCV/Data/Features.npy', features)
# np.save('OpenCV/Data/Labels.npy', labels)

Testing Model (Detecting Faces)

In [21]:
img = cv.imread('Data/Images/testFaces/find8.jpeg')

haar = cv.CascadeClassifier('Data/faceData.xml')
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read('Data/face_trained.yml')
# features = np.load('Data/Features.npy', allow_pickle=True)
# # labels = np.load('Data/Labels.npy')

tmp = gray(img)
people = [p for p in os.listdir('Data/Images/trainFaces')]

faceRect = haar.detectMultiScale(tmp, 1.1, 3)
for (x1, y1, x2, y2) in faceRect:
    face = tmp[y1:y1+y2, x1:x1+x2]
    label, _ = face_recognizer.predict(face)                            # _ (confidence) is inverse of accuracy

    cv.putText(img, people[label], (x1+13, y1+y2+23), cv.FONT_HERSHEY_COMPLEX, 0.75, (0, 255, 0), 1)
    cv.rectangle(img, (x1, y1), (x1+x2, y1+y2), (0, 255, 0), 2)

    cv.imshow('Detected', img)

if cv.waitKey(0) & 0xFF == ord('q'):
    cv.destroyAllWindows()

Note : The built-in face recognizer (one we discussed above) is not very accurate, and is not recommended for
       large projects. Also, not only do we have a limited training dataset, but also the training data is not
       very accurate. So, it is recommended to use a more advanced and accurate face recognizer, like DNN.